In [1]:
#!sudo pip install --upgrade transformers
#!sudo pip install fuzzywuzzy

     |████████████████████████████████| 7.8MB 6.8MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 86.6MB/s eta 0:00:01
  Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0


In [46]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,BitsAndBytesConfig
import json
#from Gpt4_OpenAI_Prompt import system_msg,context,postprocess
#from prompt1 import system_msg
import pandas as pd
import transformers
from fuzzywuzzy import process
print(transformers.__version__)

4.34.1


In [2]:
print(torch.cuda.is_available())
#print(torch.cuda.get_device_name())

False


In [15]:
context = {
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
                         "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
                        {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
                        {"ENTITY": "contribution_to_growth",
                         "other names": ["contribution to growth", "growth", "grown"]},
                        {"ENTITY": "kda_transactional",
                         "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
                        {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
                        {"ENTITY": "correlation",
                         "other names": ["associate", "associated", "association", "associations", "correlate",
                                         "correlated",
                                         "correlation", "correlations", "relate", "related", "relation", "relations",
                                         "relationship",
                                         "relationships"]}
                        ],
    "DATE VARIABLE": [
        {"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
}

In [45]:
def get_context_df():
    df_final = pd.DataFrame()
    for i in context:
        df = pd.DataFrame(context[i])
        df["Type"]  = i
        df_final = pd.concat([df,df_final],ignore_index=True)
    return df_final

In [ ]:
################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float32"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "fp4"

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
# Load the entire model on the GPU 0
device_map = {"": 0}


In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# model_mistral = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config,
#     device_map=device_map)
model_mistral = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer_mistral = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
def gereneate_output_mistral(prompt):
    #prompt = json.dumps(prompt)
    model_input = tokenizer_mistral(prompt, return_tensors="pt")#.to("cuda")
    #model.eval()
    response = tokenizer_mistral.decode(model_mistral.generate(**model_input, max_length=4069*2)[0], skip_special_tokens=True)
    return response

In [5]:
################################################################################
# Setting the same prompt which is used for GPT4 open AI
################################################################################

def get_user_question_for_open_ai_prompt(question):
    #return """Now Based on this following "CONTEXT" data lookup:\n""" + str(context) + "\n return a response for the question:\n" + question + "\n"
    return "\n Now detect from this question:\n" + question + "\n"

def compose_prompt_open_ai_prompt(question,base_prompt):
    return f"""[INST]<<SYS>>
            {base_prompt}
            <</SYS>>
            {get_user_question_for_open_ai_prompt(question)}
            [/INST]
            [_separator_]"""

def post_process_open_ai_prompt_response(output):
    mql = output.split("[END]")[1].split("\n\nReasoning:\n\n")[0]
    reason = output.split("[END]")[1].split("\n\nReasoning:\n\n")[1]
    print("*"*50+"MQL"+"*"*50)
    print(mql)
    print("\n"+"*"*50+"Reason"+"*"*50)
    print(reason)
    #return mql,reason

In [57]:
def post_process_model_output(output):
    output_data = (output.split("[_separator_]")[1].split("**Output**:")[1])
    lines = output_data.strip().split('\n')
    result = {}
    for line in lines:
        key, value = line.strip().split(':')
        key = key.replace("- ","")
        result[key.strip()] = value.strip()

    # Convert the dictionary to JSON
    #json_string = json.dumps(result, indent=4)
    return result

def find_best_match_from_context(word, df):
    matches = process.extractOne(word, df.values.flatten())
    max_match, score = matches
    index = df.values.flatten().tolist().index(max_match)
    row_index, col_index = divmod(index, df.shape[1])
    entity_name = df.iat[row_index, col_index]
    return df.iloc[row_index].to_dict()

In [7]:
METRIC_CONSTRAINT= """

**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATOR" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "When was the first time sales of segments was 0"

**Output**:
{"COMPARISON OPERATOR": "=",
"COMPARISON VALUE": "0"(If detected else None)
"ENTITY": "Sales" (If detected else None)}

*Note*: If the input question does not contain any comparison intent or the entity, you can indicate that they were not detected.

"""

In [10]:
ADJECTIVE_and_TONE = """
**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "least," "highest performing," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "How many top 3 segments are performing well?"

**Output**:
- ADJECTIVE: top
- TONE: positive
- ENTITY: Segment
- RANK ADJECTIVE: TOP
- TONE: positive
- RANK: 3

**Note**: If the input question doesn't contain an adjective, doesn't imply a quantity, or if it's not clear which entity the adjective should be applied to, you can indicate that they were not detected.
"""

In [14]:
FILTER_PROMPT = """**Task: Extract Filter Information**

Identify the filter entities in the user's question and set their inclusion or exclusion . For each filter entity mentioned in the question, check if it's intended to be included or excluded.

Return the following attributes for each detected filter entity:

EXCLUDE: Set to TRUE if the entity is intended to be excluded, FALSE otherwise.
ENTITY: The specific filter entity identified from the question.

If the question specifies exclusion for an entity, set EXCLUDE to TRUE; otherwise, set it to FALSE.

**Return:**
- EXCLUDE: TRUE or FALSE
- ENTITY: The filter entity"""

In [19]:
DERIVED_MEASURE = """Task: Extract the Derived Measure from User Queries

Understand the following Derived Measure definitions:
1. "Ratio": Represents a Derived Measure and can also be referred to as "ratio," "share," "contribution," "percentage," "proportion," or "contributing."
2. "Why": Represents a Derived Measure and has synonyms such as "why," "cause of," "reason for," and "diagnose."
3. "Contribution to Growth": Denotes a Derived Measure and includes synonyms like "contribution to growth," "growth," or "grown."
4. "KDA Transactional": Refers to a Derived Measure with synonyms including "kda," "key drivers," "key driver," "drivers," or "driver."
5. "Growth Rate": Represents a Derived Measure with synonyms like "growth rate," "growth," or "grown."
6. "Correlation": Denotes a Derived Measure and has synonyms such as "associate," "associated," "association," "associations," "correlate," "correlated," "correlation," "correlations," "relate," "related," "relation," "relations," "relationship," or "relationships."

Your task is to examine the user's question to identify the Derived Measure concepts. If detected, find the specific targeted derived measure mentioned in the question and return it as the Derived Measure.

Return:
- Derived Measure: (if identified)
- ENTITIES: ["ENTITY1", "ENTITY2"] (if specified in relation to the Derived Measure)
"""

In [44]:
%%time
question = "show me quantity > 100K"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(f"""Metric_constraint model output \n : {output.split("[_separator_]")[1]}""")
try:
    metric_constraint_output = json.loads(output.split("[_separator_]")[1])
except Exception as e:
    print("METRIC_CONSTRAINT model not detected any ENTITIES or Detection was not in a correct format")
    print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Metric_constraint model output 
 : 
            {"COMPARISON OPERATOR": ">",
             "COMPARISON VALUE": "100K",
             "ENTITY": "Quantity"}
            
CPU times: user 6min 58s, sys: 603 ms, total: 6min 58s
Wall time: 1min 1s


In [58]:
for i in metric_constraint_output:
    if i=='ENTITY':
        return find_best_match_from_context(metric_constraint_output['ENTITY'], get_context_df())
print("Metric_Constraint Model has not detected any ENTITY")
    

{'COMPARISON OPERATOR': '>', 'COMPARISON VALUE': '100K', 'ENTITY': 'Quantity'}

In [56]:
find_best_match_from_context(word, df)

{'ENTITY': 'correlation', 'other names': ['associate', 'associated', 'association', 'associations', 'correlate', 'correlated', 'correlation', 'correlations', 'relate', 'related', 'relation', 'relations', 'relationship', 'relationships'], 'Type': 'DERIVED MEASURE', 'parent': nan}


In [51]:
(df_final.iloc[6])

{'ENTITY': 'correlation',
 'other names': ['associate',
  'associated',
  'association',
  'associations',
  'correlate',
  'correlated',
  'correlation',
  'correlations',
  'relate',
  'related',
  'relation',
  'relations',
  'relationship',
  'relationships'],
 'Type': 'DERIVED MEASURE',
 'parent': nan}

In [21]:
%%time
question = "Ratio of sales in Dubai vs india"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,DERIVED_MEASURE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



            Derived Measure: Ratio
            ENTITIES: ["Dubai", "India"]
            
CPU times: user 5min 52s, sys: 947 ms, total: 5min 53s
Wall time: 53.5 s


In [16]:
%%time
question = "sales of Phone in Dubai"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




            **Task: Extract Filter Information**

Identify the filter entities in the user's question and set their inclusion or exclusion . For each filter entity mentioned in the question, check if it's intended to be included or excluded.

Return the following attributes for each detected filter entity:

EXCLUDE: Set to TRUE if the entity is intended to be excluded, FALSE otherwise.
ENTITY: The specific filter entity identified from the question.

If the question specifies exclusion for an entity, set EXCLUDE to TRUE; otherwise, set it to FALSE.

**Return:**
- EXCLUDE: TRUE or FALSE
- ENTITY: The filter entity
            [/ _separator_]

            **Filter Entities:**

            - ENTITY: Phone
            - ENTITY: Dubai

            **Filter Inclusion:**

            - EXCLUDE: FALSE
            - ENTITY: Phone
            - EXCLUDE: FALSE
            - ENTITY: Dubai
CPU times: user 8min 40s, sys: 808 ms, total: 8min 41s
Wall time: 1min 8s


In [17]:
question = "sales of segements except in US"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,FILTER_PROMPT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




            **Task: Extract Filter Information**

Identify the filter entities in the user's question and set their inclusion or exclusion . For each filter entity mentioned in the question, check if it's intended to be included or excluded.

Return the following attributes for each detected filter entity:

EXCLUDE: Set to TRUE if the entity is intended to be excluded, FALSE otherwise.
ENTITY: The specific filter entity identified from the question.

If the question specifies exclusion for an entity, set EXCLUDE to TRUE; otherwise, set it to FALSE.

**Return:**
- EXCLUDE: TRUE or FALSE
- ENTITY: The filter entity
            [/separator_]

            **Filter Entities:**

            - sales
            - segments
            - US

            **Filter Information:**

            - sales: EXCLUDE: FALSE, ENTITY: sales
            - segments: EXCLUDE: FALSE, ENTITY: segments
            - US: EXCLUDE: TRUE, ENTITY: US


In [49]:
%%time
question = "show me quantities > 100K"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATOR" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "show me quantities > 100K"

**Output**:
- COMPARISON OPERATOR: greater than
- COMPAR

In [50]:
%%time
question = "When is first time discount of phone is lesser than 0"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,METRIC_CONSTRAINT))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract "COMPARISON OPERATOR," "COMPARISON VALUE," and the Targeted ENTITY

**Instructions**:

1. Read the input question carefully.
2. Determine if there is a comparison intent present, such as "greater than," "less than," or "equals to." etc, If there is a comparison intent, please proceed to extract the relevant information.
3. Extract the "COMPARISON OPERATORS" based on the detected intent. Examples of "COMPARISON OPERATORS" include "greater than," "less than," and "equals to."
4. Identify and extract the "COMPARISON VALUE" from the sentence. This should be a numerical or scalar value.
5. Detect the ENTITY (e.g., "Sales") on which these comparisons are applied. Please extract the relevant entity.
6. Provide the following information:
   - COMPARISON OPERATOR: (If detected else None) 
   - COMPARISON VALUE: (If detected else None)
   - ENTITY: (If detected else None)

**Input Sentence**: "When is first time discount of phone is lesser than 0"

**Output**:
- COMPARISON OP

In [11]:
%%time
question = "show me the 2 top segments basis sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "least," "highest performing," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "show me the 2 top segments basis sales"

**Output**:
- ADJECTIVE: top
- TONE: positive
- ENTITY: Segment
- RANK ADJECTIVE: TOP
- TONE: positive
- RANK: 2
CPU times: user 24.9 s, sys: 744 ms, total: 25.6 s

In [8]:
%%time
question = "top 2 segments and bottom 3 sub-category basis quantity"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)




**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "top," "bottom," "best," "worst," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "top 2 segments and bottom 3 sub-category basis quantity"

**Output**:
- ADJECTIVE: top, bottom
- TONE: mixed (positive and negative)
- ENTITY: Segment, sub-category
- RANK ADJECTIVE: TOP, BOTTOM
- TONE

In [12]:
%%time
question = "show me the 2 top segments basis sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)




**Task**: Extract Adjectives, Determine Intent (TONE), Identify Entity, Return RANK ADJECTIVE, and Scalar (RANK)

**Instructions**:

1. Carefully read the user question to identify any adjectives present. Adjectives can include terms like "least," "highest performing," etc.
2. Determine the intent (TONE) of the adjective. The intent can be either positive or negative.
   - If the adjective intent is negative, always set RANK ADJECTIVE to "BOTTOM" and TONE to "negative."
   - If the adjective intent is positive, always  set RANK ADJECTIVE to "TOP" and TONE to "positive."
3. If the adjective implies a quantity (e.g., "how many," "top 3"), extract that scalar value as RANK.
4. Identify the entity (e.g., "Segment") on which the detected adjective should be applied.

**Input Question**: "show me the 2 top segments basis sales"

**Output**:
- ADJECTIVE: top
- TONE: positive
- ENTITY: Segment
- RANK ADJECTIVE: TOP
- TONE: positive
- RANK: 2
CPU times: user 18.8 s, sys: 602 ms, total: 19.4 s

In [14]:
ADJECTIVE_and_TONE_output = post_process_model_output(output)
ADJECTIVE_and_TONE_output

{'ADJECTIVE': 'top',
 'TONE': 'positive',
 'ENTITY': 'Segment',
 'RANK ADJECTIVE': 'TOP',
 'RANK': '2'}

In [12]:
{
    "DIMENSION": {
        "segments": [
            {
                "ENTITY": "Segment",
                "RANK": [
                    {
                        "RANK ADJECTIVE": "top",
                        "RANK VALUE": "2"
                    }
                ]
            }
        ]
    },
    "MEASURE": {
        "sales": [
            {
                "ENTITY": "Sales"
            }
        ]
    }
}


{'DIMENSION': {'segments': [{'ENTITY': 'Segment',
    'RANK': [{'RANK ADJECTIVE': 'top', 'RANK VALUE': '2'}]}]},
 'MEASURE': {'sales': [{'ENTITY': 'Sales'}]}}

In [16]:
#!pip install fuzzywuzzy

In [11]:
%%time
question = "top 2 and bottom 3 segments by sales"
output=gereneate_output_mistral(compose_prompt_open_ai_prompt(question,ADJECTIVE_and_TONE))
#post_process_open_ai_prompt_response(output)
print(output.split("[_separator_]")[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2507: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 15.60 GiB total capacity; 14.56 GiB already allocated; 14.94 MiB free; 14.91 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF